Jupyter Notebook: Spotify Track Prediction Model by Joseph Kim (6-26-2024)
Data taken from Kaggle

In [ ]:
# begin by importing all packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorboard as tf
import os 

import warnings
warnings.filterwarnings('ignore')

import the data in via os
view the data imported in and put into pandas dataframes
preprocessing (sklearn packages)
    encoding to change explicit (false, true) into numerical values (0, 1)
    scaling to scale the data to standardized values (difference in distribution between variables for separation of colors)
use a correlation matrix and outlier plots
supervised learning - classification model for genre, graph with nearest neighbors
linear regression model for determining popularity

unsupervised learning - dimensionality reduction for PCA, tSNE, UMAP
clustering by 150 genres
